---
# Imports

In [19]:
import pandas as pd
import missingno as msno

---
# Load the data

In [20]:
df = pd.read_csv('../data/new_df.csv')

C:\Users\Nikola\AppData\Local\Temp\ipykernel_60748\23254591.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/new_df.csv')


In [21]:
df.head(3)

,SensorId,Stamp,humidity,pm10,pm25,temperature,Hour,Month_Day_Calculation
0,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:00:00+00:00,86.0,55.0,30.0,4.0,0.00,1.032258
1,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:15:00+00:00,86.0,59.5,32.0,4.0,0.25,1.032258
2,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:30:00+00:00,86.0,64.0,34.0,4.0,0.50,1.032258


In [22]:
len(df)

5217076

---
# Data Preparation

Drop columns due to need for further forecasting

In [23]:
df = df.drop(columns=['humidity', 'temperature', 'pm25'])

## Deal with missing values

In [24]:
df.isnull().sum() / len(df) * 100

SensorId                 0.000000
Stamp                    0.000000
pm10                     0.044719
Hour                     0.000000
Month_Day_Calculation    0.000000
dtype: float64

### Interpolation

In [25]:
df['pm10'] = df['pm10'].interpolate(method='cubic', limit_direction='both')

In [26]:
df.isnull().sum() / len(df) * 100

SensorId                 0.0
Stamp                    0.0
pm10                     0.0
Hour                     0.0
Month_Day_Calculation    0.0
dtype: float64

## Shift the target variable

In [27]:
df.head(3)

,SensorId,Stamp,pm10,Hour,Month_Day_Calculation
0,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:00:00+00:00,55.0,0.00,1.032258
1,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:15:00+00:00,59.5,0.25,1.032258
2,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:30:00+00:00,64.0,0.50,1.032258


In [28]:
df['pm10 - 1'] = df['pm10'].shift(1)
df['pm10 - 2'] = df['pm10'].shift(2)
df['pm10 - 3'] = df['pm10'].shift(3)

In [29]:
df['SensorId  - 1'] = df['SensorId'].shift(1)
df['SensorId  - 2'] = df['SensorId'].shift(2)
df['SensorId  - 3'] = df['SensorId'].shift(3)

In [30]:
len(df)

5217076

In [31]:
df = df[df['SensorId'] == df['SensorId  - 1']]
df = df[df['SensorId'] == df['SensorId  - 2']]
df = df[df['SensorId'] == df['SensorId  - 3']]

In [32]:
len(df)

5216641

In [33]:
df.head(3)

,SensorId,Stamp,pm10,Hour,Month_Day_Calculation,pm10 - 1,pm10 - 2,pm10 - 3,SensorId - 1,SensorId - 2,SensorId - 3
3,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:45:00+00:00,68.0,0.75,1.032258,64.0,59.5,55.0,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5
4,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:00:00+00:00,65.0,1.00,1.032258,68.0,64.0,59.5,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5
5,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:15:00+00:00,54.0,1.25,1.032258,65.0,68.0,64.0,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5,007f2b03-94e6-47b3-9e3e-44273354acd5


## Data Cleaning

In [35]:
df = df.drop(columns=['SensorId  - 1', 'SensorId  - 2', 'SensorId  - 3'])

In [42]:
df

,SensorId,Stamp,pm10,Hour,Month_Day_Calculation,pm10 - 1,pm10 - 2,pm10 - 3
3,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 00:45:00+00:00,68.0,0.75,1.032258,64.0,59.5,55.0
4,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:00:00+00:00,65.0,1.00,1.032258,68.0,64.0,59.5
5,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:15:00+00:00,54.0,1.25,1.032258,65.0,68.0,64.0
6,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:30:00+00:00,55.0,1.50,1.032258,54.0,65.0,68.0
7,007f2b03-94e6-47b3-9e3e-44273354acd5,2023-01-01 01:45:00+00:00,56.0,1.75,1.032258,55.0,54.0,65.0
...,...,...,...,...,...,...,...,...
5217071,sensor_dev_8550_72,2024-11-19 13:45:00+00:00,0.0,13.75,11.633333,0.0,0.0,0.0
5217072,sensor_dev_8550_72,2024-11-19 14:00:00+00:00,0.0,14.00,11.633333,0.0,0.0,0.0
5217073,sensor_dev_8550_72,2024-11-19 14:15:00+00:00,0.0,14.25,11.633333,0.0,0.0,0.0
5217074,sensor_dev_8550_72,2024-11-19 14:30:00+00:00,0.0,14.50,11.633333,0.0,0.0,0.0


---
# Save the new dataframe

In [43]:
df.to_csv('../data/interpolated_without_nearest.csv', index=False)